In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rcParams["figure.figsize"] = (15, 5)
from pmdarima import auto_arima, model_selection, ARIMA
from pmdarima.arima.utils import ndiffs
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from datetime import datetime, timedelta, date
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import joblib

In [2]:
countries = 'Spain|Denmark'
datecol = 'date'
exog=pd.read_csv("data/exogenous.csv", parse_dates=[datecol], index_col=[datecol])
endog=pd.read_csv("data/endogenous.csv", parse_dates=[datecol], index_col=[datecol])
cases="cases"

for cases in ["cases","deaths"]:

    for country in countries.split('|'):
        #filename = './models/' + country + 'SARIMAXmodel.pkl'
        filename = './models/' + country + '_new_'+cases+'_model.pkl'
        model = joblib.load(filename)
        if model.fittedvalues.index[-1]== exog.index[-1]:
            print("up to date")
        else:

            exogmodel=exog.loc[:, exog.columns.str.contains(country)]
            endogmodel=endog.loc[:, endog.columns.str.contains("new_{}_{}".format(cases,country))]
            #exogmodel=exogmodel.iloc[-1:,:]
            sc_in = MinMaxScaler(feature_range=(0, 1))
            scaled_input = sc_in.fit_transform(exogmodel)
            scaled_input = pd.DataFrame(scaled_input, index=exogmodel.index, columns=exogmodel.columns)
            X_update = scaled_input
            X_update=X_update.iloc[-1:,:]

            sc_out = MinMaxScaler(feature_range=(0, 1))
            scaled_output = sc_out.fit_transform(endogmodel)
            scaled_output  = pd.Series(scaled_output.flatten(), index=endogmodel.index, name=endogmodel.columns[0])
            y = scaled_output.resample('1D').sum()
            y=y.iloc[-1:]
            modelupdated=model.append(y, exog=X_update, refit=True,)
            joblib.dump(modelupdated, filename)
            print("{} new_{} updated".format(country,cases))

ValueError: Given `endog` does not have an index that extends the index of the model.

In [3]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:        new_cases_Spain   No. Observations:                  261
Model:               SARIMAX(2, 2, 0)   Log Likelihood                 728.014
Date:                Tue, 22 Sep 2020   AIC                          -1422.027
Time:                        22:47:51   BIC                          -1361.561
Sample:                    01-01-2020   HQIC                         -1397.716
                         - 09-17-2020                                         
Covariance Type:                  opg                                         
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
positive_rate_Spain                            0.1693      0.075      2.265      0.024       0.023       0.316
C1_School closing_Spain                        0.0081      0.128      0.064      0.949      -0.242       0.258
C2_Workplace closing_Spain                     0.0101      0.156      0.065      0.948      -0.296       0.316
C3_Cancel public events_Spain                  0.1792      0.317      0.566      0.572      -0.442       0.800
C4_Restrictions on gatherings_Spain           -0.0426      0.110     -0.387      0.699      -0.258       0.173
C5_Close public transport_Spain               -0.0670      0.091     -0.739      0.460      -0.245       0.111
C6_Stay at home requirements_Spain             0.0913      0.045      2.048      0.041       0.004       0.179
C7_Restrictions on internal movement_Spain     0.0026      0.121      0.022      0.983      -0.234       0.239
C8_International travel controls_Spain        -0.2321      0.408     -0.569      0.569      -1.031       0.567
H1_Public information campaigns_Spain         -0.0514      3.614     -0.014      0.989      -7.134       7.031
H2_Testing policy_Spain                       -0.1070      0.075     -1.420      0.156      -0.255       0.041
H3_Contact tracing_Spain                       0.0223      3.604      0.006      0.995      -7.041       7.086
mask_Spain                                     1.3771      0.582      2.365      0.018       0.236       2.518
flightsArr_Spain                               0.0807      0.276      0.293      0.770      -0.460       0.622
ar.L1                                         -0.6124      0.047    -13.144      0.000      -0.704      -0.521
ar.L2                                         -0.3226      0.035     -9.201      0.000      -0.391      -0.254
sigma2                                         0.0002   1.17e-05     18.221      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       85.78   Jarque-Bera (JB):              1002.34
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):              21.09   Skew:                             1.25
Prob(H) (two-sided):                  0.00   Kurtosis:                        12.31
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:        new_cases_Spain   No. Observations:                  260
Model:               SARIMAX(1, 1, 3)   Log Likelihood                 728.540
Date:                Thu, 17 Sep 2020   AIC                          -1419.080
Time:                        00:05:06   BIC                          -1351.501
Sample:                    01-01-2020   HQIC                         -1391.909
                         - 09-16-2020                                         
Covariance Type:                  opg                                         
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
positive_rate_Spain                            0.1592      0.073      2.180      0.029       0.016       0.302
C1_School closing_Spain                       -0.0323      0.145     -0.222      0.824      -0.317       0.253
C2_Workplace closing_Spain                     0.0248      0.139      0.179      0.858      -0.247       0.297
C3_Cancel public events_Spain                  0.2537      0.287      0.884      0.377      -0.309       0.816
C4_Restrictions on gatherings_Spain           -0.1257      0.108     -1.160      0.246      -0.338       0.087
C5_Close public transport_Spain               -0.0607      0.109     -0.559      0.576      -0.274       0.152
C6_Stay at home requirements_Spain             0.0834      0.062      1.344      0.179      -0.038       0.205
C7_Restrictions on internal movement_Spain     0.0448      0.118      0.380      0.704      -0.186       0.276
C8_International travel controls_Spain        -0.3177      0.375     -0.848      0.396      -1.052       0.416
H1_Public information campaigns_Spain          0.0036      1.586      0.002      0.998      -3.105       3.112
H2_Testing policy_Spain                       -0.1138      0.066     -1.730      0.084      -0.243       0.015
H3_Contact tracing_Spain                      -0.0288      1.550     -0.019      0.985      -3.066       3.009
mask_Spain                                     0.7295      0.510      1.431      0.153      -0.270       1.729
flightsArr_Spain                              -0.0831      0.290     -0.286      0.775      -0.652       0.486
ar.L1                                          0.8871      0.039     22.679      0.000       0.810       0.964
ma.L1                                         -0.6325      0.066     -9.577      0.000      -0.762      -0.503
ma.L2                                          0.0201      0.063      0.317      0.751      -0.104       0.144
ma.L3                                          0.1178      0.053      2.230      0.026       0.014       0.221
sigma2                                         0.0002   1.13e-05     18.193      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       72.59   Jarque-Bera (JB):              1172.53
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):              10.29   Skew:                             1.60
Prob(H) (two-sided):                  0.00   Kurtosis:                        12.92
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""